In [1]:
!pip install --upgrade certifi

In [2]:
import urllib.request
import json
import ssl

In [3]:
from dmtools_client_package.dmtools_client_module import DMToolsClient

In [4]:
import certifi

In [5]:
urllib.request.__version__

'3.12'

In [10]:
## get api key and user id from local .env file
from dotenv import load_dotenv
import os
from os import environ, path
#BASE_DIR = path.abspath(path.dirname(__file__))
BASE_DIR = os.getcwd()
load_dotenv(path.join(BASE_DIR, ".env"))
MY_DMTOOLS_APIKEY = environ.get("MY_DMTOOLS_APIKEY")
MY_DMTOOLS_USERID = environ.get("MY_DMTOOLS_USERID")
Client = DMToolsClient(MY_DMTOOLS_USERID, MY_DMTOOLS_APIKEY)

In [11]:
BASE_DIR

'/home/andrew1tb/Documents/Code/Work/Brown_Working/dmtools_notebooks/Demos'

In [7]:
api_server = "https://dmtools.brown.edu/"
data_api = "dmtool/fastapi_data/open/data/"
fastapi_url_data = api_server + data_api + "update_a_data/?id_in=" + str(1)

In [8]:
request_header = {'dmtool-userid': MY_DMTOOLS_USERID,
 'dmtool-apikey': MY_DMTOOLS_APIKEY,
 'Content-Type': 'application/x-www-form-urlencoded'}
dmtools_userid = MY_DMTOOLS_USERID
dmtools_apikey = MY_DMTOOLS_APIKEY

In [9]:
purpose = 'read_a'
subject = 'data'
id = 1
Client.create_current_url(purpose, subject, id)
a_data_json_obj = Client.read_current_as_json()

HTTPError: HTTP Error 400: Bad Request

In [ ]:

context = ssl._create_unverified_context()

# The nested JSON data as a Python list (will be converted to string)
nested_data = [[['1', '1'], ['1', '0']], [['1', '1'], ['1', '0']]]
simple_data = "data"
# Convert the nested data to a JSON string
nested_data_string = json.dumps(nested_data)

# Construct the payload with the nested JSON string
payload = {
    "data": nested_data_string
}

json_data = json.dumps(payload)

payload["data"]

# Convert the payload to a JSON string and encode it to bytes
#json_data = json.dumps(payload).encode('utf-8')

# Create an SSL context (if needed)


# Create the request
#request = urllib.request.Request(fastapi_url_data, data=json_data, headers=request_header, )

#request = urllib.request.Request(fastapi_url_data, headers=request_header, method='PATCH')
#r = urllib.request.urlopen(request, context=context)
#string = r.read().decode('utf-8')



In [8]:
def update_current(data_in,url_in):
    if url_in == '':
        encoded_data = data_in.encode('utf-8')
        create_request = urllib.request.Request(self.current_url, data=encoded_data, method='PATCH')
        create_request.add_header('dmtool-userid', str(dmtools_userid))
        create_request.add_header('dmtool-apikey', dmtool_apikey)
        #create_request.add_header('Content-Type', 'application/x-www-form-urlencoded')
        create_request.add_header('Content-Type', 'application/json')
    else:
        encoded_data = data_in.encode('utf-8')
        create_request = urllib.request.Request(url_in, data=encoded_data, method='PATCH')
        create_request.add_header('dmtool-userid', str(dmtools_userid))
        create_request.add_header('dmtool-apikey', dmtools_apikey)
        create_request.add_header('Content-Type', 'application/json')
        #create_request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    
    try:
        with urllib.request.urlopen(create_request, context=context) as response:
            response_data = response.read().decode('utf-8')
            response_json_obj = json.loads(response_data)
            return response_json_obj
    except urllib.error.HTTPError as e:
        if e.code == 307:
            redirect_url = e.headers['Location']
            return create_current(data_in, redirect_url)
        elif e.code == 422:
            print("HTTP 422 Unprocessable Entity")
            error_response = e.read().decode('utf-8')
            print("Response content:", error_response)
        else:
            print(f"Error: {e.code}")
            error_response = e.read().decode('utf-8')
            print("Response content:", error_response)

In [9]:
update_current(json_data, fastapi_url_data)

{'updated': {'Data': 'experiment=\'Theory\' y_rescale=\' \' date_of_run_start=None public=1 spin_dependency=\'SI\' default_color=\'LtR\' date_of_run_end=None open=1 id=1 result_type=\'Th\' default_style=\'Fill\' data=\'[[["1", "1"], ["1", "0"]], [["1", "1"], ["1", "0"]]]\' date_of_announcement=datetime.date(2003, 1, 1) official=1 measurement_type=\'Dir\' data_values=\'{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;630 3.2e-44;640 3e-44;640 1.2e-44;630 1e-44;225 9e-45;215 7e-45;315 7e-46;470 2.1e-46;650 7e-47;660 6.8e-47;660 2e-47;650 2e-47;470 6e-47;315 1e-46;215 3e-46;100 8e-46;60 1e-45;60 4e-43]}\' data_label=\'Baer et. al., 2003, constraints on mSUGRA, SI\' date_official=None greatest_hit=0 nomhash=\'--- \\nknikolic: false\\n\' data_comment=\' \' created_at=None x_units=\'GeV\' data_reference=\'JCAP 0309 (2003) 007 , hep-ph/0305191\' year=2003 updated_at=datetime.datetime(2013, 9, 6, 8, 39, 30) limit_id=1 y_units=\'cm^2\' file_name=\'Baer_hep-ph0305191.limit\' rating=0 archived_at=date